In [20]:
import matplotlib.pyplot as plt 
import json
import pickle
import torch
import random 
import numpy as np
import os 

from torch.optim import SGD, Adam
from torch.utils.data import DataLoader, random_split
from torch.nn.functional import binary_cross_entropy
from sklearn import metrics 

from data_loaders.assist2009 import ASSIST2009
from data_loaders.assist2012 import ASSIST2012
from data_loaders.ednet01 import EdNet01

from models.dkvmn_text import SUBJ_DKVMN
from models.dkvmn_text import train_model as plus_train

from models.utils import collate_fn, collate_ednet, cal_acc_class

%matplotlib inline

seed = 1004

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

In [21]:
model_name = 'dkvmn+'
dataset_name = 'ASSIST2009'
dataset = None
ckpts = f"ckpts/{model_name}/{dataset_name}/"

with open("config.json") as f:
    config = json.load(f)
    model_config = config[model_name]
    train_config = config["train_config"]
    
batch_size = train_config["batch_size"]
num_epochs = train_config["num_epochs"]
train_ratio = train_config["train_ratio"]
learning_rate = train_config["learning_rate"]
optimizer = train_config["optimizer"] # can be sgd, adam
seq_len = train_config["seq_len"] # 샘플링 할 갯수

In [22]:
# 데이터셋 추가 가능
collate_pt = collate_fn
if dataset_name == "ASSIST2009":
    dataset = ASSIST2009(seq_len, 'datasets/ASSIST2009/')
elif dataset_name == "ASSIST2012":
    dataset = ASSIST2012(seq_len, 'datasets/ASSIST2012/')

In [23]:
def train_model(model, test_loader, ckpt_path):
    '''
        Args:
            train_loader: the PyTorch DataLoader instance for training
            test_loader: the PyTorch DataLoader instance for test
            num_epochs: the number of epochs
            opt: the optimization to train this model
            ckpt_path: the path to save this model's parameters
    '''
    aucs = []
    loss_means = []  
    accs = []
    q_accs = {}
    
    max_auc = 0
    
    # Test
    model.load_state_dict(torch.load(os.path.join(ckpt_path, "model.ckpt"), map_location=device))
    loss_mean = []
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            q, r, qshft_seqs, rshft_seqs, m, bert_s, bert_t, bert_m, q2diff_seqs, pid_seqs, pidshift, hint_seqs = data

            model.eval()

            y, Mv, w = model(q.long(), r.long(), bert_s, bert_t, bert_m, q2diff_seqs.long(), pid_seqs.long())

            # y와 t 변수에 있는 행렬들에서 마스킹이 true로 된 값들만 불러옴
            q = torch.masked_select(q, m).detach().cpu()
            y = torch.masked_select(y, m).detach().cpu()
            t = torch.masked_select(r, m).detach().cpu()

            auc = metrics.roc_auc_score(
                y_true=t.numpy(), y_score=y.numpy()
            )
            bin_y = [1 if p >= 0.5 else 0 for p in y.numpy()]
            acc = metrics.accuracy_score(t.numpy(), bin_y)
            loss = binary_cross_entropy(y, t) # 실제 y^T와 원핫 결합, 다음 answer 간 cross entropy

            print(f"[Test] number: {i}, AUC: {auc}, ACC: :{acc} Loss: {loss} ")

            # evaluation metrics
            aucs.append(auc)
            loss_mean.append(loss)     
            accs.append(acc)
            q_accs, cnt = cal_acc_class(q.long(), t.long(), bin_y)
        loss_means.append(np.mean(loss_mean))


    return aucs, loss_means, accs, q_accs, cnt, Mv, w

In [24]:
model = torch.nn.DataParallel(SUBJ_DKVMN(dataset.num_q, num_qid=dataset.num_pid, **model_config)).to(device)
train_model = train_model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
# 데이터셋 분할
data_size = len(dataset)
train_size = int(data_size * train_ratio) 
valid_size = int(data_size * ((1.0 - train_ratio) / 2.0))
test_size = data_size - train_size - valid_size

train_dataset, valid_dataset, test_dataset = random_split(
    dataset, [train_size, valid_size, test_size], generator=torch.Generator(device=device)
)

# pickle에 얼마만큼 분할했는지 읽기
if os.path.exists(os.path.join(dataset.dataset_dir, "train_indices.pkl")):
    with open(
        os.path.join(dataset.dataset_dir, "train_indices.pkl"), "rb"
    ) as f:
        train_dataset.indices = pickle.load(f)
    with open(
        os.path.join(dataset.dataset_dir, "valid_indicies.pkl"), "rb"
    ) as f:
        valid_dataset.indices = pickle.load(f)
    with open(
        os.path.join(dataset.dataset_dir, "test_indices.pkl"), "rb"
    ) as f:
        test_dataset.indices = pickle.load(f)

In [26]:
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    collate_fn=collate_pt, generator=torch.Generator(device=device)
)
valid_loader = DataLoader(
    valid_dataset, batch_size=batch_size, shuffle=True,
    collate_fn=collate_pt, generator=torch.Generator(device=device)
)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True,
    collate_fn=collate_pt, generator=torch.Generator(device=device)
)

if optimizer == "sgd":
    opt = SGD(model.parameters(), learning_rate, momentum=0.9)
elif optimizer == "adam":
    opt = Adam(model.parameters(), learning_rate)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.5)
opt.lr_scheduler = lr_scheduler

# 모델에서 미리 정의한 함수로 AUCS와 LOSS 계산    
print(iter(DataLoader(test_loader)))
aucs, loss_means, accs, q_accs, q_cnts, Mv, w = \
    train_model(
        model, test_loader, ckpts
    )

[Test] number: 0, AUC: 0.9646951885545804, ACC: :0.9255774165953806 Loss: 0.19650167226791382 
[Test] number: 1, AUC: 0.937318381042593, ACC: :0.9030303030303031 Loss: 0.2672674357891083 
[Test] number: 2, AUC: 0.9574817518248175, ACC: :0.9142441860465116 Loss: 0.19510900974273682 
[Test] number: 3, AUC: 0.9638503273134823, ACC: :0.9406028368794326 Loss: 0.16736353933811188 
[Test] number: 4, AUC: 0.9740740274877779, ACC: :0.935813953488372 Loss: 0.17980289459228516 
[Test] number: 5, AUC: 0.9332156147945622, ACC: :0.8936170212765957 Loss: 0.32247358560562134 
[Test] number: 6, AUC: 0.9448115445778187, ACC: :0.905103668261563 Loss: 0.26523008942604065 
[Test] number: 7, AUC: 0.9251589766341319, ACC: :0.9076433121019108 Loss: 0.28238198161125183 
[Test] number: 8, AUC: 0.9464691393452423, ACC: :0.9108159392789373 Loss: 0.25684311985969543 
[Test] number: 9, AUC: 0.9504729243321068, ACC: :0.8993288590604027 Loss: 0.2788538932800293 
[Test] number: 10, AUC: 0.9709116510483458, ACC: :0.927

In [34]:
print(torch.max(Mv[:, :-1])) # 컨셉 수 / 시퀀스? 임베딩 수

tensor(4.9477)


In [28]:
print(w)

tensor([[[1.9650e-03, 2.6068e-04, 8.4568e-04,  ..., 1.0739e-03,
          8.8093e-04, 3.0276e-04],
         [1.9650e-03, 2.6068e-04, 8.4568e-04,  ..., 1.0739e-03,
          8.8093e-04, 3.0276e-04],
         [1.9650e-03, 2.6068e-04, 8.4568e-04,  ..., 1.0739e-03,
          8.8093e-04, 3.0276e-04],
         ...,
         [3.8164e-04, 4.8337e-04, 2.9430e-04,  ..., 2.5905e-04,
          4.3908e-04, 2.0913e-04],
         [3.8164e-04, 4.8337e-04, 2.9430e-04,  ..., 2.5905e-04,
          4.3908e-04, 2.0913e-04],
         [3.8164e-04, 4.8337e-04, 2.9430e-04,  ..., 2.5905e-04,
          4.3908e-04, 2.0913e-04]],

        [[3.5993e-03, 9.4661e-04, 6.0555e-04,  ..., 6.3319e-04,
          2.9606e-04, 9.4043e-04],
         [3.5993e-03, 9.4661e-04, 6.0555e-04,  ..., 6.3319e-04,
          2.9606e-04, 9.4043e-04],
         [1.0867e-03, 5.8854e-04, 1.8075e-03,  ..., 2.2399e-04,
          1.9610e-04, 2.8893e-04],
         ...,
         [1.1919e-03, 1.1772e-03, 1.2758e-03,  ..., 1.8060e-03,
          2.179

In [75]:
# 모든 W값에 대한 빈 텐서를 생성
# 마지막 layer 빼서 넘겨보기

# Read Process = w * M
# 그래서 R 값은 마스크한 형태이고, W값은 마스킹 한 쪽에 0값 달라는 얘기인 것 같음
# 그럼 결론적으로 메모리만 들어가면 됨
# Memory 

a = torch.zeros_like(w)
print(Mv[:, :-1].shape)
a[12][99][4] = 1
print(torch.concat([a.unsqueeze(-1), Mv[:, :-1]], dim=-1).sum(-2).shape)
print(SUBJ_DKVMN(dataset.num_q, num_qid=dataset.num_pid, **model_config).k_emb_layer(q))
print(SUBJ_DKVMN(dataset.num_q, num_qid=dataset.num_pid, **model_config).f_layer((torch.concat([a, torch.zeros_like(Mv[:, :-1])], dim=-1).sum(-2).T * torch.concat([a.unsqueeze(-1), Mv[:, :-1]], dim=-1).sum(-2))))


print(w[0][99][4])
print()


torch.Size([13, 100, 100, 100])
torch.Size([13, 100, 101])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: Tensors must have same number of dimensions: got 3 and 4